# Feature engineering -- Normal mode analyses

## Imports

In [ ]:
# imports
import nmafeat
from prody import *
import numpy as np

## Compute normal modes

In [3]:
nmafeat.computeNMDfile()
(coords, modes) = nmafeat.extractModesCoords()
print(f"Values range from {coords.min()} to {coords.max()}")

@> PDB file is found in working directory (2k39.pdb.gz).
@> 76 atoms and 116 coordinate set(s) were parsed in 0.27s.
@> Starting iterative superposition:
@> Step #1: RMSD difference = 5.4124e-01
@> Step #2: RMSD difference = 2.1728e-04
@> Step #3: RMSD difference = 2.3341e-07
@> Iterative superposition completed in 0.27s.
@> Covariance is calculated using 116 coordinate sets.
@> Covariance matrix calculated in 0.016614s.
@> 20 modes were calculated in 0.10s.


Values range from 10.672 to 37.841


In [15]:
# %%
dmin = coords.min() - 2
dmax = coords.max() + 2
a = np.mgrid[dmin:dmax:101j,dmin:dmax:101j,dmin:dmax:101j]
a = a.reshape(3,1030301).transpose()

## Interpolate the data onto the grid

In [ ]:
# %%
import scipy.interpolate.ndgriddata as ndgriddata
mode1_interp = ndgriddata.griddata(coords, modes[0], a)     # interpolate the data onto the grid
# %%
# Eureka! Maybe it worked!
isnotnan = mode1_interp[~np.isnan(mode1_interp)].size
isnan = mode1_interp[np.isnan(mode1_interp)].size

## Compute the gradient of the interpolated data

In [16]:
δ = np.gradient(mode1_interp)
δ[0][~np.isnan(δ[0])].size

532695

In [17]:
# Compute the first modes' size, then create the gradient and check its' size
m1i_orig = mode1_interp.reshape(3,101,101,101)
m1i_grad = np.gradient(m1i_orig)
m1i_grad[0].shape

(3, 101, 101, 101)

In [18]:
# find where the gradient is zero
np.where(m1i_grad[0] == 0)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

## Compute singularity polynomial and dimensions

In [ ]:
# 